In [2]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI(model_name="gpt-3.5-turbo-1106")

# a = llm.predict("how many planets are there?")
b = chat.predict("how many planets are there?")
print(b)


There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Additionally, there is a debate about the classification of Pluto as a planet, with some considering it the ninth planet and others classifying it as a dwarf planet.


In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
chat = ChatOpenAI(temperature=0.1)

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in korean."),
    AIMessage(content="안녕 은채야!"),
    HumanMessage(
        content="What is the distance between seoul and Paris. Also, what is your name?",),
]

chat.predict_messages(messages)


AIMessage(content='서울과 파리 사이의 거리는 약 9,500km입니다. 제 이름은 비밀이에요.')

In [5]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mextico", country_b="Thailand")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line. However, the actual distance may vary depending on the specific locations within each country and the mode of transportation used.'

In [16]:
# 변수 관리형 개선 템플릿 코드

template = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human",
         "What is the distance between {country_a} and {country_b}. Also, what is your name?",),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="celina",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Κλειώ. Πώς μπορώ να βοηθήσω;')

In [19]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()
p.parse("Hello ,how , are,you")


['Hello', 'how', 'are', 'you']

In [26]:
template = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT replay with anything else"),
        ("human",
         "{question}",),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question=" What are the planets"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)


['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(model_name="gpt-3.5-turbo-1106")


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT replay with anything else"),
        ("human",
         "{question}",),
    ]
)
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})


['Pikachu', 'Bulbasaur', 'Charizard', 'Squirtle', 'Jigglypuff']

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


# chef
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a world-class international chef. You create easy to follow resipies for any type of cuisine with easy to find ingredients"),
        ("human",
         "I want to cook {cuisine} food.",),
    ]
)
chef_chain = chef_prompt | chat

# veg_chef
veg_chef = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. you don't radically modify th recipe. If there is no alternative for a food just say you don't know how to replace it."),

        ("human",
         "{recipe}",),
    ]
)
veg_chef_chain = veg_chef | chat

# final
final_chain = {"recipe": chef_chain} | veg_chef_chain

final_chain.invoke({
    "cuisine": "indian"
})


Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's make a classic dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) tomato sauce
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1/2 tsp turmeric
- 1/2 tsp paprika
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tbsp vegetable oil, half of the minced garlic, half of the grated ginger, and a pinch of salt. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. 

AIMessageChunk(content="As a vegetarian chef, I can help you make a vegetarian version of Chicken Tikka Masala by replacing the chicken with a suitable alternative. In this case, you can use paneer, a type of Indian cottage cheese that is commonly used in vegetarian dishes. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) tomato sauce\n- 1 tbsp garam masala\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1/2 tsp turmeric\n- 1/2 tsp paprika\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the paneer in the yogurt, lemon juice, vegetable oil, garlic, gi